## A minimalist example for recovering sparse graphs using `uGLAD`

Fitting uGLAD on a erdos-renyi random sparse graph with samples obtained from a corresponding multivariate Gaussian distribution.    

### About `uGLAD` 
An unsupervised learning based approach to recover sparse graphs. This work proposes `uGLAD` which is a unsupervised version of a previous `GLAD` model (GLAD: Learning Sparse Graph Recovery (ICLR 2020 - [link](<https://openreview.net/forum?id=BkxpMTEtPB>)).  

Key benefits & features:  
- Solution to Graphical Lasso: A better alternative to solve the Graphical Lasso problem as
    - The neural networks of the uGLAD enable adaptive choices of the hyperparameters which leads to better performance than the existing algorithms  
     - No need to pre-specify the sparsity related regularization hyperparameters    
    - Requires less number of iterations to converge due to neural network based acceleration of the unrolled optimization algorithm (Alternating Minimization)    
    - GPU based acceleration can be leveraged  
- Glasso loss function: The loss is the logdet objective of the graphical lasso `1/M(-1*log|theta|+ <S, theta>)`, where `M=num_samples, S=input covariance matrix, theta=predicted precision matrix`.  
- Ease of usability: Matches the I/O signature of `sklearn GraphicalLassoCV`, so easy to plug-in to the existing code.  

In [1]:
import os, sys
# reloads modules automatically before entering the 
# execution of code typed at the IPython prompt.
%load_ext autoreload
%autoreload 2
# install jupyter-notebook in the env if the prefix does not 
# show the desired virtual env. 
print(sys.prefix)
import warnings
warnings.filterwarnings('ignore')

/home/harshx/anaconda3/envs/uGLAD


In [2]:
import torch
torch.__version__

'1.10.1'

### Create sample data

In [3]:
from uGLAD import main
# Xb = samples batch, trueTheta = corresponding true precision matrices
Xb, true_theta = main.get_data(
    num_nodes=10, 
    sparsity=0.2, 
    num_samples=500, 
    batch_size=1,
    eig_offset=0.1, 
    w_min=0.5,
    w_max=1
)
print(f'true_theta: {true_theta.shape}, Samples {Xb.shape}')

true_theta: (1, 10, 10), Samples (1, 500, 10)


### Running the uGLAD model

Until Convergence:  
1. Initialize learnable `GLAD` parameters  
2. Run the GLAD model  
3. Get the glasso-loss  
4. Backprop  

Possible solutions if `uGLAD` does not converge:  
1. Increase number of training EPOCHS
2. Lower the learning rate    
3. Please re-run. This will run the optimization with different initializations  
4. Change the INIT_DIAG=0/1 in the `GLAD` model parameters 
5. Increase `L`, the number of unrolled iterations of `GLAD` 

In [7]:
from uGLAD import main
model_uGLAD = main.uGLAD_GL()  # Initialize the model
model_uGLAD.fit(Xb[0], true_theta[0], centered=False, epochs=250)  # fit to the data

epoch:0/250 loss:15.489583969116211
epoch:25/250 loss:14.473474502563477
epoch:50/250 loss:14.189064979553223
epoch:75/250 loss:14.107943534851074
epoch:100/250 loss:14.031806945800781
epoch:125/250 loss:13.940059661865234
epoch:150/250 loss:13.852167129516602
epoch:175/250 loss:12.969505310058594
epoch:200/250 loss:10.19554328918457
epoch:225/250 loss:8.690495491027832
Compare - {'FDR': 0.0, 'TPR': 1.0, 'FPR': 0.0, 'SHD': 0, 'nnzTrue': 12, 'nnzPred': 12, 'precsion': 1.0, 'recall': 1.0, 'Fbeta': 1.0, 'aupr': 1.0, 'auc': 1.0}


In [8]:
print(model_uGLAD.precision_,'\n', model_uGLAD.precision_.shape)

[[ 1.4404836  -0.         -0.         -0.         -0.          0.
  -0.         -0.          0.42251015  0.38467064]
 [-0.          1.4293299  -0.         -0.          0.          0.
  -0.         -0.         -0.          0.6477385 ]
 [-0.         -0.          1.5232587  -0.         -0.         -0.
   0.          0.          0.         -0.        ]
 [-0.         -0.         -0.          1.5234779  -0.          0.
  -0.          0.25111294 -0.          0.43402958]
 [-0.          0.         -0.         -0.          1.4153395   0.49050713
  -0.         -0.          0.20844766  0.        ]
 [ 0.          0.         -0.          0.          0.4905072   1.3864069
   0.20581746 -0.          0.30376875  0.        ]
 [-0.         -0.          0.         -0.         -0.          0.20581758
   1.6625088  -0.          0.4712029  -0.        ]
 [-0.         -0.          0.          0.25111333 -0.         -0.
  -0.          1.2687237   0.56496006  0.2649657 ]
 [ 0.42251015 -0.          0.         -0.

In [9]:
print(true_theta[0])

[[1.80510005 0.         0.         0.         0.         0.
  0.         0.         0.6079522  0.74880283]
 [0.         1.80510005 0.         0.         0.         0.
  0.         0.         0.         0.90568032]
 [0.         0.         1.80510005 0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         1.80510005 0.         0.
  0.         0.61633679 0.         0.83926166]
 [0.         0.         0.         0.         1.80510005 0.88125476
  0.         0.         0.56310128 0.        ]
 [0.         0.         0.         0.         0.88125476 1.80510005
  0.60941677 0.         0.74112496 0.        ]
 [0.         0.         0.         0.         0.         0.60941677
  1.80510005 0.         0.76290138 0.        ]
 [0.         0.         0.         0.61633679 0.         0.
  0.         1.80510005 0.76953828 0.66825042]
 [0.6079522  0.         0.         0.         0.56310128 0.74112496
  0.76290138 0.76953828 1.80510005 0.        ]
 [0.7